In [ ]:
import sys
print(f'Using python version {sys.version}') # get python version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/247_project_data'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

import torch # import pytorch
import torch.nn as nn # pytorch's neural networks module
import torch.nn.functional as F # functional module.

print(f'Using pytorch version {torch.__version__}')

In [ ]:
print(torch.cuda.is_available())
device_id = 0 if torch.cuda.is_available() else 'cpu' # Equivalent to device_id = 'cuda:0'
device = torch.device(device_id) # use these semantics to specify a specific device. 

Prepare the data

In [ ]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

In [ ]:
X_train_valid[0]

In [ ]:
# Convert to 0-4 labeling and integer type (integers needed for categorical labels)
y_train_valid = (y_train_valid - np.min(y_train_valid)).astype('int')
y_test = (y_test - np.min(y_test)).astype('int')

In [ ]:
#Find the subject 1
Subject1_train_index = np.count_nonzero(person_train_valid == 0)
Subject1_test_index = np.count_nonzero(person_test == 0)
Sub1_train_X = X_train_valid[0:Subject1_train_index]
Sub1_train_y = y_train_valid[0:Subject1_train_index]
Sub1_test_X = X_test[0:Subject1_test_index]
Sub1_test_y = y_test[0:Subject1_test_index]
print('Sub1_train_X:',Sub1_train_X.shape)
print('Sub1_train_y:',Sub1_train_y.shape)
print('Sub1_test_X:',Sub1_test_X.shape)
print('Sub1_test_y:',Sub1_test_y.shape)

## Build the CNN

In [ ]:
# inherits from nn.Module
class DeepConvNet(nn.Module):
    def __init__(self, input_shape=(22, 1000), n_temporal_filters=40, n_spatial_filters=40, n_classes=4):
        super().__init__() # call __init__ method of superclass
        
        self.conv1 = nn.Conv2d(1, 25, kernel_size=(1,10), stride=1, padding=0)
        self.conv2 = nn.Conv2d(25, 25, kernel_size=(22,1), stride=1, padding=0)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(1,3), stride=3)
        self.conv3 = nn.Conv2d(25, 50, kernel_size=(1,10), stride=1, padding=0)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(1,3), stride=3)
        self.conv4 = nn.Conv2d(50, 100, kernel_size=(1,10), stride=1, padding=0)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(1,3), stride=3)
        self.conv5 = nn.Conv2d(100, 200, kernel_size=(1,10), stride=1, padding=0)
        self.maxpool4 = nn.MaxPool2d(kernel_size=(1,3), stride=3)
        self.dense = nn.LazyLinear(n_classes)
        return
    
    # declaring a forward method also makes the instance a callable.
    # e.g.:
    # model = ShallowConvNet()
    # out = model(x)
    def forward(self, x):
      x = x.view(-1, 1, 22, 1000)
      #x = F.normalize(x)
      x = F.elu(self.conv1(x))
      x = F.elu(self.conv2(x))
      x = self.maxpool1(x)
      x = F.elu(self.conv3(x))
      x = self.maxpool2(x)
      x = F.elu(self.conv4(x))
      x = self.maxpool3(x)
      x = F.elu(self.conv5(x))
      x = self.maxpool4(x)
      x = x.view(x.shape[0], -1)
      x = self.dense(x)
      return x
       

In [ ]:
class ShallowConvNet(nn.Module):
    def __init__(self, input_shape=(22, 1000), n_temporal_filters=40, n_spatial_filters=40, n_classes=4, use_BN = False):
        super().__init__() # call __init__ method of superclass
        
        self.input_shape = input_shape # last two dimensions, (excluding batch size). Should be length 2.
        self.n_temporal_filters = n_temporal_filters
        self.n_spatial_filters = n_spatial_filters
        self.n_classes = n_classes
        self.use_BN = use_BN

        self.temporal_convolution = nn.Conv2d(1, n_temporal_filters, (1, 25))
        self.spatial_convolution = nn.Conv2d(n_temporal_filters, n_spatial_filters, (input_shape[0], 1))
        self.average_pool = nn.AvgPool2d((1, 75), stride=(1, 15))
        self.BN_temporal = nn.BatchNorm2d(num_features = n_temporal_filters)
        self.BN_spatial = nn.BatchNorm2d(num_features = n_spatial_filters)

        # Final linear layer
        self.n_dense_features = n_spatial_filters*(1 + ((input_shape[1] - 25 + 1) - 75) // 15)
        self.dense = nn.Linear(self.n_dense_features, n_classes)
        # you can also use 'self.dense = nn.LazyLinear(n_classes)' to avoid having to manually compute features

        self.elu = nn.ELU()
        self.dropout = nn.Dropout()
        return
    
    # declaring a forward method also makes the instance a callable.
    # e.g.:
    # model = ShallowConvNet()
    # out = model(x)
    def forward(self, x):
        # x has shape (batch_size, input_shape[0], input_shape[1])
        # Let H0 = input_shape[0], H1 = input_shape[1]
        h = x

        # note that h.view(-1, 1, h.shape[1], h.shape[2]) works normally but does not work with torchinfo
        # this is because the torchinfo input has a weird shape
        h = h.view(-1, 1, self.input_shape[0], self.input_shape[1]) # view as (batch_size, 1, input_shape[0], input_shape[1])
        # Sometimes, view doesn't work and you have to use reshape. This is because of how tensors are stored in memory.

        # 2d convolution takes inputs of shape (batch_size, num_channels, H, W)
        h = self.temporal_convolution(h) # (batch_size, 1, H0, W0) -> (batch_size, n_temporal_filters, H0, W0 - 25 + 1)
        if self.use_BN == True:
          h = self.BN_temporal(h)
        h = self.elu(h)

        h = self.spatial_convolution(h) # (batch_size, n_temporal_filters, H0, W0 - 25 + 1) -> (batch_size, n_spatial_filters, 1, W0 - 25 + 1)
        if self.use_BN == True:
          h = self.BN_spatial(h)
        h = self.elu(h)

        h = h**2 # square
        # alternatively, use torch.pow(h, 2.0)

        h = self.average_pool(h) # (batch_size, n_spatial_filters, 1, W0 - 25 + 1) -> (batch_size, n_spatial_filters, 1, 1 + ((W0 - 25 + 1) - 75)//15)

        h = torch.log(h) # (natural) log

        h = h.view(h.shape[0], -1) # flatten the non-batch dimensions
        h = self.dense(h) # (batch_size, self.n_dense_features) -> (batch_size, n_classes)
        #h = self.dropout(h)

        return h


In [ ]:
# torchsummary is deprecated. Use torchinfo instead (https://github.com/TylerYep/torchinfo).
!pip install torchinfo

In [ ]:
from torchinfo import summary
# Only uses outputs of modules.
print(summary(ShallowConvNet(use_BN = True), input_size=(22, 1000)))
#print(summary(DeepConvNet(), input_size=(22, 1000)))

## Build Dataset

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        if isinstance(X, np.ndarray):
            self.X = torch.FloatTensor(X) # 32-bit float
        else:
            self.X = X
        if isinstance(Y, np.ndarray):
            self.Y = torch.LongTensor(Y) # integer type
        else:
            self.Y = Y
        return
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [ ]:
# Prepare the pretrain dataset with only subject 1 data, no validation
Sub1_train_X = torch.FloatTensor(Sub1_train_X).to(device)
Sub1_train_y = torch.LongTensor(Sub1_train_y).to(device)
Sub1_test_X = torch.FloatTensor(Sub1_test_X).to(device)
Sub1_test_y = torch.LongTensor(Sub1_test_y).to(device)

batch_size = 256
train_dataset_subject1 = MyDataset(Sub1_train_X, Sub1_train_y)
train_dataloader_subject1 = torch.utils.data.DataLoader(train_dataset_subject1, batch_size, shuffle=True)
Test_dataset_subject1 = MyDataset(Sub1_test_X,Sub1_test_y)
Test_dataloader_subject1 = torch.utils.data.DataLoader(Test_dataset_subject1, batch_size, shuffle=False)


In [ ]:
len(train_dataset_subject1), len(train_dataloader_subject1)

In [ ]:
# Split the wholedata into training and validation sets
from sklearn.model_selection import train_test_split
X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(X_train_valid, y_train_valid, test_size=0.2)
X_train = torch.FloatTensor(X_train_np).to(device)
X_val = torch.FloatTensor(X_val_np).to(device)
y_train = torch.LongTensor(y_train_np).to(device)
y_val = torch.LongTensor(y_val_np).to(device)

In [ ]:
batch_size = 256
train_dataset = MyDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False)

In [ ]:
#Test
X_test = torch.FloatTensor(X_test).to(device)
y_test = torch.LongTensor(y_test).to(device)
Test_dataset = MyDataset(X_test,y_test)
Test_dataloader = torch.utils.data.DataLoader(Test_dataset, batch_size, shuffle=False)

In [ ]:
len(train_dataset), len(train_dataloader)

Train a Model

In [ ]:
# choose which model to use
model = ShallowConvNet(use_BN = True).to(device)
#model = DeepConvNet().to(device)

In [ ]:
import tqdm # for progress bar
# Set the random seeds.
# Sometimes some operations are not deterministic.
# See https://pytorch.org/docs/stable/notes/randomness.html
#   and https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html
torch.manual_seed(12345) # set the random seed for pytorch
random.seed(12345) # just in case
np.random.seed(12345) # just in case

# For weight regularization, add weight_decay=l2_regularization_strength to the optimizer instantiation,
#   or implement it manually and add it to the loss before backward propagation.

# Declare an optimizer that handles weight updates
# This optimizer also has its own state dictionary
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay = 0.05)
celoss = nn.CrossEntropyLoss()

In [ ]:
#Pretrain on Subject1
loss_hist_subject1 = []
acc_hist_subject1 = []
test_acc_hist_subject1 = []

num_epochs = 35

# progress bar
pbar = tqdm.tqdm(
    range(num_epochs), position=0, leave=True,
    bar_format='{l_bar}{bar:30}{r_bar}',
)
for epoch in pbar:
    model.train() # set model to training mode.
    for batch in train_dataloader_subject1:
        optimizer.zero_grad() # clear gradients of parameters that optimizer is optimizing
        x, y = batch

        # x = x.to(device) # necessary if X is not on the same device as model
        # y = y.to(device)

        out = model(x) # shape (batch_size, n_classes)
        loss = celoss(out, y) # calculate the cross entropy loss

        loss.backward() # backpropagate
        optimizer.step() # perform optimization step

        # IMPORTANT: DO NOT store 'loss' by itself, since it references its entire computational graph.
        # Otherwise you will run out of memory.
        # You MUST use .item() to convert to a scalar or call .detach().
        loss_hist_subject1.append(loss.item())
    
    model.eval() # set model to evaluation mode. Relevant for dropout, batchnorm, layernorm, etc.
    # calculate accuracy for training and validation sets
    ns = 0 # number of samples
    nc = 0 # number of correct outputs
    with torch.no_grad():
        for batch in train_dataloader_subject1:
            x, y = batch
            out = model(x)
            ns += len(y)
            nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
    acc_hist_subject1.append(nc/ns)

    # update progress bar postfix
    pbar.set_postfix({'acc': acc_hist_subject1[-1]})

In [ ]:
test_loss_hist_subject1 = []
test_acc_hist_subject1 = []
ns = 0
nc = 0
model.eval()
with torch.no_grad():
    for batch in Test_dataloader_subject1:
        x, y = batch
        out = model(x)
        loss = celoss(out, y)
        test_loss_hist_subject1.append(loss.item())
        ns += len(y)
        nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
test_acc_hist_subject1.append(nc/ns)
print('Test Subject1 ACC:',test_acc_hist_subject1[-1])

In [ ]:
test_loss_hist_subject1_all = []
test_acc_hist_subject1_all = []
ns = 0
nc = 0
model.eval()
with torch.no_grad():
    for batch in Test_dataloader:
        x, y = batch
        out = model(x)
        loss = celoss(out, y)
        test_loss_hist_subject1_all.append(loss.item())
        ns += len(y)
        nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
test_acc_hist_subject1_all.append(nc/ns)
print('Test ACC:',test_acc_hist_subject1_all[-1])

In [ ]:
#Save the pretrain model
torch.save(model.state_dict(), 'subject1_pretrain.pt')

In [ ]:
# Load weight
model = ShallowConvNet(use_BN = True).to(device)
chkpt = torch.load('subject1_pretrain.pt')
model.load_state_dict(chkpt)

In [ ]:
loss_hist = []
val_loss_hist = []
acc_hist = []
val_acc_hist = []
num_epochs = 40

# progress bar
pbar = tqdm.tqdm(
    range(num_epochs), position=0, leave=True,
    bar_format='{l_bar}{bar:30}{r_bar}',
)
for epoch in pbar:
    model.train() # set model to training mode.
    for batch in train_dataloader:
        optimizer.zero_grad() # clear gradients of parameters that optimizer is optimizing
        x, y = batch

        # x = x.to(device) # necessary if X is not on the same device as model
        # y = y.to(device)

        out = model(x) # shape (batch_size, n_classes)
        loss = celoss(out, y) # calculate the cross entropy loss

        loss.backward() # backpropagate
        optimizer.step() # perform optimization step

        # IMPORTANT: DO NOT store 'loss' by itself, since it references its entire computational graph.
        # Otherwise you will run out of memory.
        # You MUST use .item() to convert to a scalar or call .detach().
        loss_hist.append(loss.item())
    
    model.eval() # set model to evaluation mode. Relevant for dropout, batchnorm, layernorm, etc.
    # calculate accuracy for training and validation sets
    ns = 0 # number of samples
    nc = 0 # number of correct outputs
    with torch.no_grad():
        for batch in train_dataloader:
            x, y = batch
            out = model(x)
            ns += len(y)
            nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
    acc_hist.append(nc/ns)

    ns = 0 # number of samples
    nc = 0 # number of correct outputs
    with torch.no_grad():
        for batch in val_dataloader:
            x, y = batch
            out = model(x)
            loss = celoss(out, y)
            val_loss_hist.append(loss.item())
            ns += len(y)
            nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
    val_acc_hist.append(nc/ns)


    # update progress bar postfix
    pbar.set_postfix({'acc': acc_hist[-1], 'val_acc': val_acc_hist[-1]})

In [ ]:
plt.figure(dpi=100)
plt.plot(loss_hist, label='training')
plt.yscale('log')
plt.ylabel('loss')
plt.legend(loc='lower left')
plt.xlabel('step (training)')
plt.twiny()
plt.plot(val_loss_hist, 'r', label='validation')
plt.xlabel('step (validation)')
plt.legend(loc='upper left')



In [ ]:
plt.figure(dpi=100)
plt.plot(acc_hist, label='training')
plt.ylabel('acc')
plt.legend(loc='lower left')
plt.xlabel('step (training)')
plt.twiny()
plt.plot(val_acc_hist, 'r', label='validation')
plt.xlabel('step (validation)')
plt.legend(loc='upper left')

In [ ]:
test_loss_hist = []
test_acc_hist = []
ns = 0
nc = 0
model.eval()
with torch.no_grad():
    for batch in Test_dataloader:
        x, y = batch
        out = model(x)
        loss = celoss(out, y)
        test_loss_hist.append(loss.item())
        ns += len(y)
        nc += (out.max(1)[1] == y).detach().cpu().numpy().sum()
test_acc_hist.append(nc/ns)
print('Test ACC:',test_acc_hist[-1])